In [ ]:
import trimesh
import numpy as np
import open3d as o3d

# ─── 1. Load your meshes ────────────────────────────────────────────────────────
lod_4_mesh = trimesh.load_mesh("/home/jeans/win/aaaJAIST/resources/LOD_data_50/1/lod4.obj")
lod_3_mesh = trimesh.load_mesh("/home/jeans/win/aaaJAIST/resources/LOD_data_50/1/lod3.obj")

# ─── 2. Sample points on the surfaces ──────────────────────────────────────────
#    (so we can run correspondence-based methods on point-clouds)
num_samples = 20000
pts_ref, _ = trimesh.sample.sample_surface(lod_4_mesh, num_samples)
pts_src, _ = trimesh.sample.sample_surface(lod_3_mesh, num_samples)

# ─── 3. Build Open3D point clouds & estimate normals ───────────────────────────
pcd_ref = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts_ref))
pcd_ref.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))

pcd_src = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts_src))
pcd_src.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))

# ─── 4. Compute FPFH features ───────────────────────────────────────────────────
radius_feature = 0.1
fpfh_ref = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_ref,
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
)
fpfh_src = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_src,
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
)

# ─── 5. Coarse alignment via RANSAC + FPFH ─────────────────────────────────────
distance_threshold = 0.2  # adjust based on your model’s scale
result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    pcd_src, pcd_ref,
    fpfh_src, fpfh_ref,
    mutual_filter=True,
    max_correspondence_distance=distance_threshold,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n=4,
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4_000_000, 500)
)
print("RANSAC result:")
print(result_ransac)

# ─── 6. Fine‐tune with Point‐to‐Plane ICP ───────────────────────────────────────
icp_threshold = 0.05
result_icp = o3d.pipelines.registration.registration_icp(
    pcd_src, pcd_ref,
    icp_threshold,
    init=result_ransac.transformation,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
print("ICP refinement:")
print(result_icp)

# ─── 7. Apply the final 4×4 transform to your trimesh LOD-3 mesh ───────────────
lod_3_mesh.apply_transform(result_icp.transformation)

# ─── 8. (Optional) Tint them differently for clarity ───────────────────────────
lod_4_mesh.visual.face_colors = [200, 200, 250, 200]  # bluish, semi‐opaque
lod_3_mesh.visual.face_colors = [250, 200, 200, 150]  # reddish, more transparent

# ─── 9. Visualize both in one scene ────────────────────────────────────────────
scene = trimesh.Scene([lod_4_mesh, lod_3_mesh])
scene.show()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
